## Exploratory Analysis on Cars data


In [4]:
%%pyspark
data_path = spark.read.load('abfss://azureudemeycoursewsfs@azureudemeycoursewsadls.dfs.core.windows.net/cars_dataset_details.parquet', format='parquet')
data_path.show(100)

+--------------------+----+-------------+---------+------+-----------+------------+--------------------+
|                name|year|selling_price|km_driven|  fuel|seller_type|transmission|               owner|
+--------------------+----+-------------+---------+------+-----------+------------+--------------------+
|       Maruti 800 AC|2007|        60000|    70000|Petrol| Individual|      Manual|         First Owner|
|Maruti Wagon R LX...|2007|       135000|    50000|Petrol| Individual|      Manual|         First Owner|
|Hyundai Verna 1.6 SX|2012|       600000|   100000|Diesel| Individual|      Manual|         First Owner|
|Datsun RediGO T O...|2017|       250000|    46000|Petrol| Individual|      Manual|         First Owner|
|Honda Amaze VX i-...|2014|       450000|   141000|Diesel| Individual|      Manual|        Second Owner|
|Maruti Alto LX BSIII|2007|       140000|   125000|Petrol| Individual|      Manual|         First Owner|
|Hyundai Xcent 1.2...|2016|       550000|    25000|Petr

In [5]:
#converting spark dataframe to pandas
df = data_path.select("*").toPandas()
df

name  year  ... transmission         owner
0                           Maruti 800 AC  2007  ...       Manual   First Owner
1                Maruti Wagon R LXI Minor  2007  ...       Manual   First Owner
2                    Hyundai Verna 1.6 SX  2012  ...       Manual   First Owner
3                  Datsun RediGO T Option  2017  ...       Manual   First Owner
4                   Honda Amaze VX i-DTEC  2014  ...       Manual  Second Owner
...                                   ...   ...  ...          ...           ...
4335  Hyundai i20 Magna 1.4 CRDi (Diesel)  2014  ...       Manual  Second Owner
4336           Hyundai i20 Magna 1.4 CRDi  2014  ...       Manual  Second Owner
4337                  Maruti 800 AC BSIII  2009  ...       Manual  Second Owner
4338     Hyundai Creta 1.6 CRDi SX Option  2016  ...       Manual   First Owner
4339                     Renault KWID RXT  2016  ...       Manual   First Owner

[4340 rows x 8 columns]
/home/trusted-service-user/cluster-env/env/lib/pytho

In [6]:
# Checking missing and null values
df.isnull().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
dtype: int64

In [7]:
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta
df['Years2Date'] = pd.to_datetime(df['year'].map(str), format='%Y-%m-%d %H:%M:%S')
df['CurrentYear'] = pd.to_datetime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')


In [8]:
df['TotalOwnershipinDays'] =pd.to_timedelta(df['CurrentYear']-df['Years2Date'], unit='D')
df['TotalOwnershipinDays']= df['TotalOwnershipinDays'].astype(str)
df[['Days','Time','Etc']] = df.TotalOwnershipinDays.str.split(" ",expand=True,)
df['Age']=df['Days'].map(int)/365
df

name  year  ...              Etc        Age
0                           Maruti 800 AC  2007  ...  21:16:22.744627  13.616438
1                Maruti Wagon R LXI Minor  2007  ...  21:16:22.744627  13.616438
2                    Hyundai Verna 1.6 SX  2012  ...  21:16:22.744627   8.613699
3                  Datsun RediGO T Option  2017  ...  21:16:22.744627   3.608219
4                   Honda Amaze VX i-DTEC  2014  ...  21:16:22.744627   6.610959
...                                   ...   ...  ...              ...        ...
4335  Hyundai i20 Magna 1.4 CRDi (Diesel)  2014  ...  21:16:22.744627   6.610959
4336           Hyundai i20 Magna 1.4 CRDi  2014  ...  21:16:22.744627   6.610959
4337                  Maruti 800 AC BSIII  2009  ...  21:16:22.744627  11.613699
4338     Hyundai Creta 1.6 CRDi SX Option  2016  ...  21:16:22.744627   4.610959
4339                     Renault KWID RXT  2016  ...  21:16:22.744627   4.610959

[4340 rows x 15 columns]

In [9]:
%%pyspark
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
sdf = spark.createDataFrame(df)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.

In [10]:
%%pyspark
sdf.registerTempTable("car_prices_details_with_age_in_years")

In [11]:
%%sql
SELECT * FROM car_prices_details_with_age_in_years